In [1]:
import numpy as np
import pandas as pd
from joblib import dump
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
seed = 48

In [3]:
tar_features = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']
tar_sd_features = ['X4_sd', 'X11_sd', 'X18_sd', 'X50_sd', 'X26_sd', 'X3112_sd']
log_features = ['X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

In [4]:
train = pd.read_csv('./data/train.csv', index_col='id').fillna(0)
train = train + 1e-5

In [5]:
x_feature = train.columns[:-12]
list(x_feature)

['WORLDCLIM_BIO1_annual_mean_temperature',
 'WORLDCLIM_BIO12_annual_precipitation',
 'WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month',
 'WORLDCLIM_BIO15_precipitation_seasonality',
 'WORLDCLIM_BIO4_temperature_seasonality',
 'WORLDCLIM_BIO7_temperature_annual_range',
 'SOIL_bdod_0.5cm_mean_0.01_deg',
 'SOIL_bdod_100.200cm_mean_0.01_deg',
 'SOIL_bdod_15.30cm_mean_0.01_deg',
 'SOIL_bdod_30.60cm_mean_0.01_deg',
 'SOIL_bdod_5.15cm_mean_0.01_deg',
 'SOIL_bdod_60.100cm_mean_0.01_deg',
 'SOIL_cec_0.5cm_mean_0.01_deg',
 'SOIL_cec_100.200cm_mean_0.01_deg',
 'SOIL_cec_15.30cm_mean_0.01_deg',
 'SOIL_cec_30.60cm_mean_0.01_deg',
 'SOIL_cec_5.15cm_mean_0.01_deg',
 'SOIL_cec_60.100cm_mean_0.01_deg',
 'SOIL_cfvo_0.5cm_mean_0.01_deg',
 'SOIL_cfvo_100.200cm_mean_0.01_deg',
 'SOIL_cfvo_15.30cm_mean_0.01_deg',
 'SOIL_cfvo_30.60cm_mean_0.01_deg',
 'SOIL_cfvo_5.15cm_mean_0.01_deg',
 'SOIL_cfvo_60.100cm_mean_0.01_deg',
 'SOIL_clay_0.5cm_mean_0.01_deg',
 'SOIL_clay_100.200cm_mean_0.01_d

In [6]:
train['X11_mean'] + train['X11_sd'] * np.random.normal(0, 0.1, len(train))

id
192027691    11.758018
195542235    15.738413
196639184     5.291262
195728812     9.454143
195251545    14.376733
               ...    
190558785    11.572787
194523231     6.099318
195888987     5.522242
135487319     7.024229
146608105    13.167113
Length: 55489, dtype: float64

In [7]:
for mean, sd in zip(tar_features, tar_sd_features):
    print(mean, sd)
    train[mean] = train[mean] + train[sd] * np.random.normal(0, 0.1, len(train))

X4_mean X4_sd
X11_mean X11_sd
X18_mean X18_sd
X50_mean X50_sd
X26_mean X26_sd
X3112_mean X3112_sd


In [8]:
train[tar_features]

,X4_mean,X11_mean,X18_mean,X50_mean,X26_mean,X3112_mean
id,,,,,,
192027691,0.400514,11.720529,0.117101,1.836432,1.235367,49.798513
195542235,0.480481,15.785940,0.389174,1.354395,0.642145,572.644888
196639184,0.796927,5.291259,8.552917,2.343163,0.395250,1130.096742
195728812,0.525442,9.756686,1.092030,1.142464,0.155982,1042.386318
195251545,0.412177,14.479782,0.663163,2.242373,10.869209,2387.438721
...,...,...,...,...,...,...
190558785,0.337253,11.572787,0.233700,1.608352,1.783203,969.547842
194523231,0.424875,6.125401,1.011073,2.427143,12.720032,1618.499115
195888987,0.642805,5.543114,2.699266,2.725587,10.249980,597.798903


In [9]:
scaler = StandardScaler()
y_train = np.zeros_like(train[tar_features], dtype=np.float32)

for idx, (target, tar_sd) in enumerate(zip(tar_features, tar_sd_features)):
    v = train[target].values

    if target in log_features:
        v = np.log10(v)

    y_train[:, idx] = v

y_train = scaler.fit_transform(y_train)

In [10]:
y_train = pd.DataFrame(y_train, columns=tar_features).set_index(train.index)
y_train

,X4_mean,X11_mean,X18_mean,X50_mean,X26_mean,X3112_mean
id,,,,,,
192027691,-0.698114,-0.282444,-1.306831,0.406724,-0.329006,-1.608147
195542235,-0.240794,0.188169,-0.577574,-0.209670,-0.586455,-0.123850
196639184,1.568900,-1.539324,1.298712,0.900045,-0.777402,0.289294
195728812,0.016328,-0.572274,0.048927,-0.554173,-1.143240,0.240194
195251545,-0.631413,0.051674,-0.253933,0.811034,0.526613,0.743842
...,...,...,...,...,...,...
190558785,-1.059888,-0.302492,-0.887245,0.138248,-0.184587,0.196169
194523231,-0.558793,-1.307971,0.002156,0.971333,0.588483,0.507596
195888987,0.687508,-1.465835,0.598419,1.206110,0.503533,-0.097724


In [11]:
train_xs, valid_xs = train_test_split(y_train, test_size=0.2, random_state=seed)
train_xs.shape, valid_xs.shape

((44391, 6), (11098, 6))

In [12]:
train_xs.to_csv('./data/processed/train.csv')
valid_xs.to_csv('./data/processed/valid.csv')
dump(scaler, './data/processed/scaler.joblib')

['./data/processed/scaler.joblib']

In [4]:
test = pd.read_csv('./data/test.csv')
test['file_path'] = test['id'].apply(lambda s: f'./data/test_images/{s}.jpeg')
test['jpeg_bytes'] = test['file_path'].apply(lambda fp: open(fp, 'rb').read())
test.to_pickle('./data/test.pkl')

In [7]:
train_scaler = StandardScaler()
x_train = np.zeros_like(train[x_feature], dtype=np.float32)

for idx, target in enumerate(x_feature):
    v = train[target].values

    # if target in log_x_features:
    #     v = np.log10(v)

    x_train[:, idx] = v

x_train = train_scaler.fit_transform(x_train)

In [10]:
x_train = pd.DataFrame(x_train, columns=x_feature).set_index(train.index)

In [11]:
x_train

,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,SOIL_bdod_30.60cm_mean_0.01_deg,...,VOD_X_1997_2018_multiyear_mean_m03,VOD_X_1997_2018_multiyear_mean_m04,VOD_X_1997_2018_multiyear_mean_m05,VOD_X_1997_2018_multiyear_mean_m06,VOD_X_1997_2018_multiyear_mean_m07,VOD_X_1997_2018_multiyear_mean_m08,VOD_X_1997_2018_multiyear_mean_m09,VOD_X_1997_2018_multiyear_mean_m10,VOD_X_1997_2018_multiyear_mean_m11,VOD_X_1997_2018_multiyear_mean_m12
id,,,,,,,,,,,,,,,,,,,,,
192027691,-0.308725,-0.889802,-0.554512,0.490021,0.805016,0.879735,0.460272,0.579818,0.487341,0.633997,...,0.638743,0.927605,1.040217,0.817117,0.347126,0.213268,0.384477,0.644727,0.544614,0.236286
195542235,0.355134,-1.259414,-0.936992,-0.583946,1.075379,1.615380,0.403478,0.248144,0.612104,0.505870,...,-0.768723,-0.881663,-0.942067,-1.069347,-1.022200,-1.020704,-0.940234,-0.787698,-0.648572,-0.492379
196639184,-0.042541,-0.203696,-0.648946,-1.225982,-0.412910,-0.264452,-0.562008,-0.481537,-0.573140,-0.583211,...,0.488973,0.413616,0.405373,0.406568,0.344944,0.294744,0.386658,0.516051,0.623689,0.651859
195728812,0.541083,0.539962,0.182802,-0.369732,-0.434778,-0.523488,0.176305,-0.614206,-0.198853,-0.262893,...,-0.321266,-0.438934,-0.559986,-0.635205,-0.534976,-0.460638,-0.527141,-0.260902,-0.219068,-0.193552
195251545,-1.833272,-0.687428,-0.640040,-0.583637,2.541932,2.258165,-1.470702,0.380814,-0.510759,0.185552,...,-0.128926,0.260018,0.553307,1.199778,1.516921,1.560907,1.258588,0.767458,0.616531,0.594171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190558785,0.645423,-1.058427,-0.726033,0.326497,-0.145290,0.276630,0.062718,-0.017194,0.175435,0.057425,...,-0.721367,-0.690451,-0.596856,-0.525751,-0.424501,-0.321623,-0.282681,-0.401587,-0.571184,-0.637861
194523231,-0.112469,0.508838,0.176645,-0.428751,0.423243,0.160330,0.460272,0.248144,0.424960,0.377743,...,-0.646642,-0.603014,-0.496135,-0.497384,-0.377590,-0.402016,-0.473746,-0.457392,-0.553931,-0.624018
195888987,0.021632,-0.620095,-0.212108,1.026921,-0.036127,0.180030,0.062718,0.977825,0.487341,0.698061,...,-1.581710,-1.633348,-1.668236,-1.719123,-1.754660,-1.738011,-1.750808,-1.761985,-1.816742,-1.792120


In [12]:
x_train.to_csv('./data/processed/train_x.csv')
dump(train_scaler, './data/processed/scaler_x.joblib')

['./data/processed/scaler_x.joblib']